# Example for system in Cugno et al 2014

Geometry is divided into 4 domains; two volumes and two surfaces:
- PM
- Cytosol
- ER membrane
- ER lumen

This model has a single species, Ca2+, with prescribed time-dependent fluxes at the PM and the ERM.

There are three reactions:
- Ca2+ influx at the PM
- Ca2+ removal in the cytosol (e.g. via buffering)
- Ca2+ flux into the ER  
```

In [54]:
import os

import dolfin as d
import sympy as sym
import numpy as np

from stubs import unit, config, common, mesh, model
from stubs.model_assembly import Compartment, Parameter, Reaction, Species

First, we define the various units for the inputs

In [55]:
# Aliases - base units
uM = unit.uM
um = unit.um
molecule = unit.molecule
sec = unit.sec
dimensionless = unit.dimensionless
# Aliases - units used in model
D_unit = um**2 / sec
flux_unit = uM * um / sec
vol_unit = uM
surf_unit = molecule / um**2

Next we generate the model.

In [56]:
def make_model(curRadius):
    # =============================================================================================
    # Species
    # =============================================================================================
    # name, initial concentration, concentration units, diffusion, diffusion units, compartment
    Ca = Species("Ca", 0.05, vol_unit, 1.0, D_unit, "Cyto")
    CaER = Species("CaER", 150.0, vol_unit, 6.27, D_unit, "ER") #effective D due to buffering
    # note that CaER is not required; it is passively determined by the prescribed flux into the ER
    # but for SMART to solve the system right now, it requires at least two species

    # =============================================================================================
    # Compartments
    # =============================================================================================
    # name, topological dimensionality, length scale units, marker value
    Cyto = Compartment("Cyto", 3, um, 1)
    PM = Compartment("PM", 2, um, 10)
    ER = Compartment("ER", 3, um, 2)
    ERm = Compartment("ERm", 2, um, 12)

    # =============================================================================================
    # Parameters and Reactions
    # =============================================================================================
    # Ca2+ influx at membrane
    gamma, alpha, beta = 1140.0, .0025, .002
    t = sym.symbols("t")
    pulsePM = gamma*(sym.exp(-t/alpha) - sym.exp(-t/beta))
    #pulsePM_I = gamma*(-alpha*sym.exp(-t/alpha) + beta*sym.exp(-t/beta)) # integral for preintegration
    j1pulse = Parameter.from_expression(
        "j1pulse", pulsePM, flux_unit, use_preintegration=False#, preint_sym_expr=pulsePM_I
        )
    r1 = Reaction(
        "r1",
        [],
        ["Ca"],
        param_map={"J": "j1pulse"},
        eqn_f_str="J",
        explicit_restriction_to_domain="PM",
    )
    # Ca2+ flux into the ER
    zeta, tER = 0.2, .02
    estep = lambda t,t0,m: 1 / (1+sym.exp(m*(t0-t)))
    pulseER = zeta*gamma*(sym.exp(-estep(t,tER,8000)*(t-tER)/alpha) - 
                          sym.exp(-estep(t,tER,8000)*(t-tER)/beta))
    #pulseER_I = zeta*gamma*(-alpha*sym.exp(-(t-tER)/alpha) + beta*sym.exp(-(t-tER)/beta)) # integral for preintegration
    j2pulse = Parameter.from_expression(
        "j2pulse", pulseER, flux_unit, use_preintegration=False#, preint_sym_expr=pulseER_I
        )
    r2 = Reaction(
        "r2",
        ["CaER"],
        ["Ca"],
        param_map={"J": "j2pulse"},
        eqn_f_str="J",
        explicit_restriction_to_domain="ERm",
    )
    # consumption of Ca in the cytosol
    tau = Parameter("tau", 0.05, sec)
    r3 = Reaction("r3", ["Ca"], [], param_map={"tau": "tau"},
         eqn_f_str="Ca/tau", species_map={"Ca": "Ca"})
    # scale fluxes into the ER to implicitly account for buffers
    #xi = 0.0227272727
    #r2.flux_scaling = {"CaER": xi}
    #r2.__post_init__()

    # =============================================================================================
    # Gather all parameters, species, compartments and reactions
    # =============================================================================================
    return common.sbmodel_from_locals(locals().values())

We load the model generated above, and load in the mesh we will use in this example.

In [57]:
curRadius = 0.25 # dendritic spine radius
i = 0
pc, sc, cc, rc = make_model(curRadius)

# =============================================================================================
# Create/load in mesh
# =============================================================================================
# Base mesh
domain, facet_markers, cell_markers = common.DemoSpheresMesh(curRadius, curRadius/2) #0 in second argument corresponds to no ER
# Write mesh and meshfunctions to file
os.makedirs(f"mesh_{i:03d}", exist_ok=True)
common.write_mesh(domain, facet_markers, cell_markers, filename=f"mesh_{i:03d}/DemoSphere")

# # Define solvers
parent_mesh = mesh.ParentMesh(
    mesh_filename=f"mesh_{i:03d}/DemoSphere.h5",
    mesh_filetype="hdf5",
    name="parent_mesh",
)
configCur = config.Config()
modelCur = model.Model(pc, sc, cc, rc, configCur, parent_mesh)
configCur.solver.update(
    {
        "final_t": 0.1,
        "initial_dt": 0.001,
        "time_precision": 6,
        "use_snes": True,
        "print_assembly": False,
    }
)

modelCur.initialize(initialize_solver=False)
modelCur.initialize_discrete_variational_problem_and_solver()


[2023-02-13 time=19:45:25] Time-dependent parameter j1pulse evaluated from expression. 
[2023-02-13 time=19:45:25] Time-dependent parameter j2pulse evaluated from expression. 
Info    : Meshing 1D...
Info    : [ 20%] Meshing curve 2 (Circle)
Info    : [ 70%] Meshing curve 5 (Circle)
Info    : Done meshing 1D (Wall 0.00337151s, CPU 0s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Sphere, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Sphere, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.831835s, CPU 0.820083s)
Info    : Meshing 3D...
Info    : 3D Meshing 2 volumes with 1 connected component
Info    : Tetrahedrizing 6468 nodes...
Info    : Done tetrahedrizing 6476 nodes (Wall 0.115483s, CPU 0.115602s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.4299s, CPU 0.415655s)
Info    : Found volume 4
Info    : Found volume 3
Info   

/usr/lib/python3/dist-packages/pandas/core/dtypes/cast.py:1784: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values
/root/shared/gitrepos/stubs-dev/stubs/model_assembly.py:188: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/usr/lib/python3/dist-packages/pandas/core/dtypes/cast.py:1784: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values
/root/shared/gitrepos/stubs-dev/stubs/model_assembly.py:188: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/usr/lib/python3/dist-packages/pandas/core/dtypes/cast.py:1784: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values
/root/shared/gitrepos/stubs-dev/stubs/model_assembly.py:188:

╒════╤═════════════╤═══════╤══════════════════╤═════════════╤══════════════╤════════════════╕
│    │ name        │    id │   dimensionality │   num_cells │   num_facets │   num_vertices │
╞════╪═════════════╪═══════╪══════════════════╪═════════════╪══════════════╪════════════════╡
│  0 │ parent_mesh │ 40507 │                3 │       69220 │       144506 │          14258 │
├────┼─────────────┼───────┼──────────────────┼─────────────┼──────────────┼────────────────┤
│  1 │ ERm         │ 40521 │                2 │         796 │            0 │            400 │
├────┼─────────────┼───────┼──────────────────┼─────────────┼──────────────┼────────────────┤
│  2 │ PM          │ 40527 │                2 │       12132 │        18198 │           6068 │
├────┼─────────────┼───────┼──────────────────┼─────────────┼──────────────┼────────────────┤
│  3 │ Cyto        │ 40533 │                3 │       66559 │       139582 │          14008 │
╘════╧═════════════╧═══════╧══════════════════╧═════════════

AssertionError: 

In [ ]:
# Write initial condition(s) to file
results = dict()
os.makedirs(f"resultsSphere_{i:03d}", exist_ok=True)
for species_name, species in modelCur.sc.items:
    results[species_name] = d.XDMFFile(
        modelCur.mpi_comm_world, f"resultsSphere_{i:03d}/{species_name}.xdmf"
    )
    results[species_name].parameters["flush_output"] = True
    results[species_name].write(modelCur.sc[species_name].u["u"], modelCur.t)

# Solve
while True:
    # Solve the system
    modelCur.monolithic_solve()
    # Save results for post processing
    for species_name, species in modelCur.sc.items:
        results[species_name].write(modelCur.sc[species_name].u["u"], modelCur.t)
    # End if we've passed the final time
    if modelCur.t >= modelCur.final_t:
        break


.......................................................................................................................
.......................................................................................................................
.................... [2023-02-13 time=14:32:48] Beginning time-step 1 [time=0.001000, dt=0.001000] ....................
.......................................................................................................................
.......................................................................................................................
[2023-02-13 time=14:32:48] Solving using PETSc.SNES Solver 
[2023-02-13 time=14:32:48] snes residual assemble (iter 1) finished in 0.059772 s 
[2023-02-13 time=14:32:48] snes jacobian assemble (iter 1) finished in 0.066319 s 
[2023-02-13 time=14:32:48] snes residual assemble (iter 2) finished in 0.059418 s 
[2023-02-13 time=14:32:49] snes jacobian assemble (iter 2) finished in 0.071540 s 
[2023-0